In [48]:
import numpy as np
#from numpy.linalg import norm
import os
from scipy.linalg import eigh, qr, null_space, norm
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'DejaVu Sans'
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from scipy.sparse import eye, kron, identity, csr_matrix, csc_matrix, lil_matrix, dok_matrix, issparse, coo_matrix
from scipy.sparse.linalg import eigsh, eigs, lobpcg, LinearOperator, ArpackNoConvergence
from scipy.optimize import curve_fit
from qutip import Qobj, ptrace, entropy_vn, qeye, tensor
from tqdm import tqdm
from itertools import product
from functools import reduce
import torch
import torch.optim as optim
from torch.autograd import Variable
import sympy as sp
from collections import Counter
from IPython.display import display, HTML

In [86]:
def pauli_x():
    """Pauli X matrix."""
    return np.array([[0, 1], [1, 0]])

def pauli_z():
    """Pauli Z matrix."""
    return np.array([[1, 0], [0, -1]])

def icosahedral_bonds(): #12 vertices
    """
    Defines the connectivity of a true 12-vertex icosahedral molecular structure.
    
    Returns:
        list of tuples: Each tuple (i, j) represents a bond between spin i and spin j.
    """
    bonds = [
        (0, 2), (0, 4), (0, 5), (0, 8), (0, 9),
        (1, 3), (1, 6), (1, 7), (1, 10), (1, 11),
        (2, 6), (2, 7), (2, 8), (2, 9), (3, 4),
        (3, 5), (3, 10), (3, 11), (4, 5), (4, 8),
        (4, 10), (5, 9), (5, 11), (6, 7), (6, 8),
        (6, 10), (7, 9), (7, 11), (8, 10), (9, 11)
    ]
    return bonds


def transverse_field_ising_icosahedral(N, J, h):
    """
    Constructs the Hamiltonian for the transverse field Ising model on an icosahedral molecular structure.
    
    Parameters:
        N (int): Number of spins (should match the icosahedral molecule, typically N=20).
        J (float): Interaction strength.
        h (float): Transverse field strength.
    
    Returns:
        H (scipy.sparse.csr_matrix): The Hamiltonian matrix in sparse format.
    """
    if N != 12:
        raise ValueError("Icosahedral molecules typically have N = 12 sites.")

    # Sparse identity matrix
    I = identity(2, format="csr")
    
    # Pauli matrices as sparse matrices
    X = csr_matrix(pauli_x())
    Z = csr_matrix(pauli_z())
    
    # Initialize the Hamiltonian
    H = csr_matrix((2**N, 2**N), dtype=np.float64)
    
    # Get icosahedral bonds
    bonds = icosahedral_bonds()
    
    # Interaction term: J * sigma_i^x * sigma_j^x for icosahedral connectivity
    for i, j in bonds:
        term = 1
        for k in range(N):
            if k == i or k == j:
                term = kron(term, X, format="csr")
            else:
                term = kron(term, I, format="csr")
        H += J * term
    
    # Transverse field term: -h * sigma_i^z
    for i in range(N):
        term = 1
        for j in range(N):
            if j == i:
                term = kron(term, Z, format="csr")
            else:
                term = kron(term, I, format="csr")
        H += -h * term
    
    return H

def ising_icosahedron(N, J):
    """
    Constructs the Hamiltonian for the transverse field Ising model on an icosahedral molecular structure without transverse field.
    
    Parameters:
        N (int): Number of spins (should match the icosahedral molecule, typically N=20).
        J (float): Interaction strength.
        """
    if N != 12:
        raise ValueError("Icosahedral molecules typically have N = 12 sites.")

    # Sparse identity matrix
    I = identity(2, format="csr")
    
    # Pauli matrices as sparse matrices
    X = csr_matrix(pauli_x())
    
    # Initialize the Hamiltonian
    H = csr_matrix((2**N, 2**N), dtype=np.float64)
    
    # Get icosahedral bonds
    bonds = icosahedral_bonds()
    
    # Interaction term: J * sigma_i^x * sigma_j^x for icosahedral connectivity
    for i, j in bonds:
        term = 1
        for k in range(N):
            if k == i or k == j:
                term = kron(term, X, format="csr")
            else:
                term = kron(term, I, format="csr")
        H += J * term
    
    return H

def transverse_field_icosahedral(N, h):
    """
    Constructs the Hamiltonian for the transverse field Ising model on an icosahedral molecular structure.
    
    Parameters:
        N (int): Number of spins (should match the icosahedral molecule, typically N=20).
        J (float): Interaction strength.
        h (float): Transverse field strength.
    
    Returns:
        H (scipy.sparse.csr_matrix): The Hamiltonian matrix in sparse format.
    """
    if N != 12:
        raise ValueError("Icosahedral molecules typically have N = 12 sites.")

    # Sparse identity matrix
    I = identity(2, format="csr")
    
    # Pauli matrices as sparse matrices
    Z = csr_matrix(pauli_z())
    
    # Initialize the Hamiltonian
    H = csr_matrix((2**N, 2**N), dtype=np.float64)
    
    # Get icosahedral bonds
    bonds = icosahedral_bonds()
    
    # Transverse field term: -h * sigma_i^z
    for i in range(N):
        term = 1
        for j in range(N):
            if j == i:
                term = kron(term, Z, format="csr")
            else:
                term = kron(term, I, format="csr")
        H += -h * term
    
    return H

#######################################################################################################################

'''
def partial_trace_qubit(rho, keep, dims):
    """Compute the partial trace of a density matrix of qubits."""
    keep_dims = np.prod([dims[i] for i in keep])
    trace_dims = np.prod([dims[i] for i in range(len(dims)) if i not in keep])
    rho = rho.reshape([keep_dims, trace_dims, keep_dims, trace_dims])
    return np.trace(rho, axis1=1, axis2=3).reshape([keep_dims, keep_dims])

def partial_trace_qubit_torch(rho, keep, dims):
    """Compute the partial trace of a density matrix of qubits using PyTorch."""
    keep_dims = torch.prod(torch.tensor([dims[i] for i in keep]))
    trace_dims = torch.prod(torch.tensor([dims[i] for i in range(len(dims)) if i not in keep]))
    rho = rho.view(keep_dims, trace_dims, keep_dims, trace_dims)
    # Compute the partial trace
    traced_rho = torch.zeros((keep_dims, keep_dims), dtype=rho.dtype)
    for i in range(trace_dims):
        traced_rho += rho[:, i, :, i]
    #return traced_rho.view(keep_dims, keep_dims)
    return traced_rho'''

def isket_numpy(arr):
    """
    Check if a NumPy array is a ket (column vector).

    Parameters:
    - arr: np.ndarray, the array to check.

    Returns:
    - bool, True if the array is a ket, False otherwise.
    """
    if not isinstance(arr, np.ndarray):
        raise ValueError("Input must be a NumPy array")

    shape = arr.shape

    if len(shape) == 2 and shape[1] == 1:
        return True
    else:
        return False

def ptrace_numpy(Q, sel, dims): # numpy function adapted from ptrace of qutip
    """
    Compute the partial trace of a density matrix of qubits using NumPy.

    Parameters:
    - Q: numpy object, the quantum object (density matrix or state vector).
    - sel: list of int, indices of the subsystems to keep.
    - dims: list of int, dimensions of the subsystems.

    Returns:
    - numpy object, the reduced density matrix after tracing out the specified subsystems.
    """
    # Get the dimensions of the subsystems
    rd = np.asarray(dims[0], dtype=np.int32).ravel()
    nd = len(rd)
    
    # Ensure sel is a sorted array of indices
    if isinstance(sel, int):
        sel = np.array([sel])
    else:
        sel = np.asarray(sel)
    sel = list(np.sort(sel))
    
    # Dimensions of the subsystems to keep
    dkeep = (rd[sel]).tolist()
    
    # Indices of the subsystems to trace out
    qtrace = list(set(np.arange(nd)) - set(sel))
    
    # Dimensions of the subsystems to trace out
    dtrace = (rd[qtrace]).tolist()
    
    # Reshape the density matrix or state vector
    rd = list(rd)
    if isket_numpy(Q):
        # Reshape and transpose for state vector
        vmat = (Q
                .reshape(rd)
                .transpose(sel + qtrace)
                .reshape([np.prod(dkeep), np.prod(dtrace)]))
        # Compute the reduced density matrix
        rhomat = vmat.dot(vmat.conj().T)
    else:
        # Reshape and transpose for density matrix
        rhomat = np.trace(Q
                          .reshape(rd + rd)
                          .transpose(qtrace + [nd + q for q in qtrace] +
                                     sel + [nd + q for q in sel])
                          .reshape([np.prod(dtrace),
                                    np.prod(dtrace),
                                    np.prod(dkeep),
                                    np.prod(dkeep)]))
    return rhomat


def ptrace_sparse(psi_sparse, keep, dims):
    """
    Compute the partial trace over arbitrary subsystems using sparse matrix operations.

    Args:
        psi_sparse (scipy.sparse matrix): Full density matrix of shape (D, D), where D = product(dims)
        keep (list of int): Subsystems to keep (indices, 0-indexed)
        dims (list of int): List of subsystem dimensions, e.g., [2]*n for n qubits

    Returns:
        scipy.sparse.csr_matrix: Reduced density matrix over kept subsystems
    """
    if not issparse(psi_sparse):
        raise ValueError("psi_sparse must be a scipy.sparse matrix")
    n = len(dims)
    D = np.prod(dims)
    if psi_sparse.shape != (D, D):
        raise ValueError("Density matrix shape does not match dims")
    trace = [i for i in range(n) if i not in keep]
    d_keep = np.prod([dims[i] for i in keep])
    # Prepare output
    data = []
    row_idx = []
    col_idx = []

    # Precompute bit masks
    def idx_to_bits(idx):
        return np.array(list(np.binary_repr(idx, width=n))).astype(int)
    

    psi_sparse = psi_sparse.tocoo()
    for i, j, val in zip(psi_sparse.row, psi_sparse.col, psi_sparse.data):
        bi = idx_to_bits(i)
        bj = idx_to_bits(j)


        # Only sum terms where traced-out subsystems agree
        if np.all(bi[trace] == bj[trace]):
            # Extract kept bits and convert to reduced indices
            #print('condition met for i, j:', i, j)
            i_red_bits = bi[keep]
            j_red_bits = bj[keep]
            i_red = int("".join(i_red_bits.astype(str)), 2)
            j_red = int("".join(j_red_bits.astype(str)), 2)


            data.append(val)
            row_idx.append(i_red)
            col_idx.append(j_red)
    
    return coo_matrix((data, (row_idx, col_idx)), shape=(d_keep, d_keep)).tocsr()


def isket_torch(arr):
    """
    Check if a PyTorch tensor is a ket (column vector).

    Parameters:
    - arr: torch.Tensor, the array to check.

    Returns:
    - bool, True if the array is a ket, False otherwise.
    """
    if not isinstance(arr, torch.Tensor):
        raise ValueError("Input must be a PyTorch tensor")

    shape = arr.shape

    if len(shape) == 2 and shape[1] == 1:
        return True
    else:
        return False

def ptrace_torch(Q, sel, dims): # torch function adapted from ptrace of qutip
    """
    Compute the partial trace of a density matrix of qubits using PyTorch.

    Parameters:
    - Q: torch.Tensor, the quantum object (density matrix or state vector).
    - sel: list of int, indices of the subsystems to keep.
    - dims: list of int, dimensions of the subsystems.

    Returns:
    - torch.Tensor, the reduced density matrix after tracing out the specified subsystems.
    """
    # Get the dimensions of the subsystems
    rd = torch.tensor(dims[0], dtype=torch.int32).flatten()
    nd = len(rd)
    #print("rd", rd)
    #print("nd", nd)
    
    # Ensure sel is a sorted array of indices
    if isinstance(sel, int):
        sel = torch.tensor([sel])
    else:
        sel = torch.tensor(sel)
    sel = torch.sort(sel).values.tolist()
    
    # Dimensions of the subsystems to keep
    dkeep = rd[sel].tolist()
    
    # Indices of the subsystems to trace out
    qtrace = list(set(range(nd)) - set(sel))
    
    # Dimensions of the subsystems to trace out
    dtrace = rd[qtrace].tolist()
    
    # Reshape the density matrix or state vector
    rd = rd.tolist()
    if isket_torch(Q):
        # Reshape and transpose for state vector
        reshaped_Q = Q.reshape(rd)
        #print(reshaped_Q.shape)
        transposed_Q = reshaped_Q.permute(sel + qtrace)
        #print(transposed_Q.shape)
        vmat = transposed_Q.reshape([torch.prod(torch.tensor(dkeep)), torch.prod(torch.tensor(dtrace))])
        #print(vmat.shape)
        # Compute the reduced density matrix
        rhomat = vmat @ vmat.conj().T
        #print(rhomat.shape)
    else:
        # Reshape and transpose for density matrix
        reshaped_Q = Q.reshape(rd + rd)
        #print("reshaped_Q", reshaped_Q.shape)
        transposed_Q = reshaped_Q.permute(qtrace + [nd + q for q in qtrace] + sel + [nd + q for q in sel])
        #print("transposed_Q", transposed_Q.shape)
        reshaped_transposed_Q = transposed_Q.reshape([torch.prod(torch.tensor(dtrace)), torch.prod(torch.tensor(dtrace)), torch.prod(torch.tensor(dkeep)), torch.prod(torch.tensor(dkeep))])
        #print("reshaped_transposed_Q", reshaped_transposed_Q.shape)
        #rhomat = torch.trace(reshaped_transposed_Q)
        rhomat = torch.einsum('iikl->kl', reshaped_transposed_Q)
        # Trace out the first two dimensions
        #rhomat = torch.zeros((torch.prod(torch.tensor(dkeep)), torch.prod(torch.tensor(dkeep))), dtype=Q.dtype)
        #for i in range(reshaped_transposed_Q.shape[0]):
        #    for j in range(reshaped_transposed_Q.shape[1]):
        #        rhomat += reshaped_transposed_Q[i, j, :, :]
        #print("rhomat", rhomat.shape)
    return rhomat

def entanglement_entropy(psi, subsystem, total_size):

    '''Computes the bipartite entanglement entropy of a pure state.
    
    Parameters:
    psi : np.array
        The wavefunction (state vector) of the full system.
    subsystem_size : int
        The number of qubits in subsystem A.
    total_size : int
        The total number of qubits in the system.
    
    Returns:
    float
        The von Neumann entanglement entropy S_A.'''
    
    psi_matrix =  np.outer(psi, psi.conj())

    # Compute the reduced density matrix rho_A = Tr_B(|psi><psi|)
    rho_A = ptrace_numpy(psi_matrix, subsystem, [[2]*total_size, [2]*total_size])  # Partial trace over B
    
    # Compute eigenvalues of rho_A
    eigenvalues = np.linalg.eigvalsh(rho_A)
    
    # Filter out zero eigenvalues to avoid numerical issues in log calculation
    eigenvalues = eigenvalues[eigenvalues > 0]
    
    # Compute von Neumann entropy S_A = -Tr(rho_A log rho_A)
    entropy = -np.sum(eigenvalues * np.log2(eigenvalues))
    
    return entropy

def entanglement_entropy_torch(psi, subsystem, total_size):
    """
    Computes the bipartite entanglement entropy of a pure state using PyTorch.

    Parameters:
    - psi: torch.Tensor (complex), the wavefunction (state vector) of the full system.
    - subsystem_size: int, the number of qubits in subsystem A.
    - total_size: int, the total number of qubits in the system.

    Returns:
    - torch.Tensor (scalar), the von Neumann entanglement entropy S_A.
    """

    if not isinstance(psi, torch.Tensor):
        psi = torch.tensor(psi, dtype=torch.complex64)
    
    # Ensure psi is normalized
    psi = psi / torch.norm(psi)

    # Compute the density matrix |psi><psi|
    psi_matrix = torch.outer(psi, psi.conj())

    # Compute the reduced density matrix rho_A = Tr_B(|psi><psi|)
    rho_A = ptrace_torch(psi_matrix, subsystem, [[2] * total_size, [2] * total_size])  # Partial trace over B

    #rho_A = rho_A.to(dtype=torch.float64)
    
    # Compute eigenvalues of rho_A
    eigvals = torch.linalg.eigvalsh(rho_A)

    # Filter out zero eigenvalues to avoid numerical issues in log calculation
    eigvals = eigvals[eigvals > 0]

    # Compute von Neumann entropy S_A = -Tr(rho_A log rho_A)
    entropy = -torch.sum(eigvals * torch.log2(eigvals))

    return entropy

def entanglement_entropy_qutip(psi, subsystem, total_size):
    
    # Convert the wavefunction to a QuTiP Qobj
    density_matrix = np.outer(psi, psi.conj())
    density_matrix_qobj = Qobj(density_matrix, dims=[[2]*total_size, [2]*total_size])

    rho_A = ptrace(density_matrix_qobj, subsystem)
    # Compute the von Neumann entropy S_A
    entropy = entropy_vn(rho_A, base=2)
    
    return entropy

def entanglement_entropy_np_ptrace(rdm):
    # rdm already computed and converted to numpy
    # Compute eigenvalues of rho_A
    eigenvalues = np.linalg.eigvalsh(rdm)
    
    # Filter out zero eigenvalues to avoid numerical issues in log calculation
    eigenvalues = eigenvalues[eigenvalues > 0]
    
    # Compute von Neumann entropy S_A = -Tr(rho_A log rho_A)
    entropy = -np.sum(eigenvalues * np.log2(eigenvalues))
    
    return entropy

def entanglement_entropy_torch_ptrace(rdm):

    eigvals = torch.linalg.eigvalsh(rdm)
    eigvals = eigvals[eigvals > 0]
    entropy = -torch.sum(eigvals * torch.log2(eigvals))
    return entropy


def entanglement_entropy_qutip_torch(psi, N):
    """
    Compute the von Neumann entanglement entropy using qutip.

    Parameters:
    - psi: torch.Tensor (complex), state vector of a quantum system.
    - N: int, total number of qubits.

    Returns:
    - torch.Tensor (scalar), von Neumann entropy.
    """
    # Ensure psi is normalized
    psi = psi / torch.norm(psi)

    # Convert PyTorch tensor to NumPy for QuTiP
    psi_np = psi.detach().numpy()

    rho_np = np.outer(psi_np, psi_np.conj())
    rho_qobj = Qobj(rho_np, dims=[[2] * N, [2] * N])

    rho_A = ptrace(rho_qobj, list(range(N // 2)))

    # Compute von Neumann entropy
    entropy = entropy_vn(rho_A, base=2)  # Compute in log base 2

    # Convert back to PyTorch tensor to allow gradient flow
    return torch.tensor(entropy, dtype=torch.float32, requires_grad=True)

#######################################################################################################################

# Define the linear combination function - numpy
def linear_combination_np(coeffs, psis):
    # Ensure psis are numpy tensors
    psi_np = [np.array(psi) for psi in psis]
    # Compute the linear combination in PyTorch
    psi = sum(c * psi for c, psi in zip(coeffs, psis))
    
    return psi

# Define the linear combination function - torch
def linear_combination(coeffs, psis):
    # Ensure psis are PyTorch tensors
    psis_torch = [torch.tensor(psi, dtype=torch.complex64) if not isinstance(psi, torch.Tensor) else psi for psi in psis]
    
    # Compute the linear combination in PyTorch
    psi_torch = sum(c * psi for c, psi in zip(coeffs, psis_torch))
    
    return psi_torch

# Define the linear combination function - torch but after computing the ptrace of outer products of scars
def linear_combination_outer(coeffs, outs):
    # Ensure outs are PyTorch tensors
    outs_torch = [torch.tensor(out, dtype=torch.complex64) if not isinstance(out, torch.Tensor) else out for out in outs]
    torch_coeffs = torch.tensor(coeffs, dtype=torch.complex64)

    # Compute the PyTorch tensor of out_coeffs which is the product of all possible combinations of c_i^* times c_j
    out_coeffs = torch.zeros((len(torch_coeffs), len(torch_coeffs)), dtype=torch.complex64)
    for i in range(len(torch_coeffs)):
        for j in range(len(torch_coeffs)):
            out_coeffs[i, j] = torch.conj(torch_coeffs[i]) * torch_coeffs[j]
    
    # Compute the linear combination in PyTorch
    lin_torch = sum(out_coeffs[i, j] * outs_torch[i] for i in range(len(coeffs)) for j in range(len(coeffs)))
    
    return lin_torch

######################################################

# chebyshev

def jackson_weights(m):
    """
    Jackson damping coefficients for k = 0..m.
    (You can replace this with your own implementation if you already have one.)
    """
    k = np.arange(m+1, dtype=float)
    N = m + 1.0
    # Standard Jackson kernel for Chebyshev series
    # g_k = [(N - k + 1) * cos(pi*k/(N+1)) + sin(pi*k/(N+1)) / tan(pi/(N+1))] / (N+1)
    gk = ((N - k + 1) * np.cos(np.pi * k / (N + 1.0)) +
          np.sin(np.pi * k / (N + 1.0)) / np.tan(np.pi / (N + 1.0))) / (N + 1.0)
    return gk

def chebyshev_filter_numpy(H, Emin, Emax, target_E0, m,
                           pad=0.05, use_jackson=True, rng=None):
    """
    Chebyshev cosine kernel filter, pure NumPy/SciPy version.

    Parameters
    ----------
    H : (n, n) array_like or sparse_matrix
        Real symmetric / Hermitian matrix.
    Emin, Emax : float
        Estimated spectral bounds of H.
    target_E0 : float
        Target energy where we want to focus the filter.
    m : int
        Polynomial degree.
    pad : float, optional
        Padding fraction for bounds.
    use_jackson : bool, optional
        Apply Jackson damping.
    rng : np.random.Generator, optional
        Random generator.

    Returns
    -------
    filt : ndarray, shape (n,)
        Normalized filtered vector.
    approx_E : float
        Rayleigh quotient <filt|H|filt>.
    """
    if rng is None:
        rng = np.random.default_rng()

    # 1) Padded bounds and rescaling parameters
    width  = Emax - Emin
    Emin_p = Emin - pad * width
    Emax_p = Emax + pad * width

    c = 0.5 * (Emax_p + Emin_p)
    d = 0.5 * (Emax_p - Emin_p)

    # 2) Rescaled target x0 and Chebyshev coefficients alpha_k
    x0 = (target_E0 - c) / d
    x0 = float(np.clip(x0, -0.999999, 0.999999))
    theta0 = np.arccos(x0)

    alpha = np.cos(np.arange(m+1) * theta0)
    if use_jackson:
        g = jackson_weights(m)
        alpha = alpha * g

    # Helper: matvec with Htilde = (H - c I)/d
    def Htilde_dot(v):
        Hv = H @ v   # works for dense or sparse
        return (Hv - c * v) / d

    # 3) Random start vector
    n = H.shape[0]
    v0 = rng.standard_normal(n)
    v0 /= norm(v0)

    # 4) Chebyshev recursion
    t0 = v0
    t1 = Htilde_dot(v0)

    filt = alpha[0] * t0 + alpha[1] * t1

    tkm1 = t0
    tk   = t1

    for k in range(2, m+1):
        tkp1 = 2.0 * Htilde_dot(tk) - tkm1
        filt = filt + alpha[k] * tkp1
        tkm1, tk = tk, tkp1

    # 5) Normalize and Rayleigh quotient
    filt_norm = norm(filt)
    if filt_norm == 0:
        raise RuntimeError("Filtered vector became zero; try different parameters.")
    filt /= filt_norm

    Hv = H @ filt
    approx_E = np.vdot(filt, Hv).real / np.vdot(filt, filt).real

    return filt, approx_E

def chebyshev_filter_numpy_v0(H, v0, Emin, Emax, target_E0, m,
                           pad=0.05, use_jackson=True, rng=None):
    """
    Chebyshev cosine kernel filter, pure NumPy/SciPy version.

    Parameters
    ----------
    H : (n, n) array_like or sparse_matrix
        Real symmetric / Hermitian matrix.
    v0 : (n,) array_like
        Initial vector to start the Chebyshev recursion.
    Emin, Emax : float
        Estimated spectral bounds of H.
    target_E0 : float
        Target energy where we want to focus the filter.
    m : int
        Polynomial degree.
    pad : float, optional
        Padding fraction for bounds.
    use_jackson : bool, optional
        Apply Jackson damping.
    rng : np.random.Generator, optional
        Random generator.

    Returns
    -------
    filt : ndarray, shape (n,)
        Normalized filtered vector.
    approx_E : float
        Rayleigh quotient <filt|H|filt>.
    """
    if rng is None:
        rng = np.random.default_rng()

    # 1) Padded bounds and rescaling parameters
    width  = Emax - Emin
    Emin_p = Emin - pad * width
    Emax_p = Emax + pad * width

    c = 0.5 * (Emax_p + Emin_p)
    d = 0.5 * (Emax_p - Emin_p)

    # 2) Rescaled target x0 and Chebyshev coefficients alpha_k
    x0 = (target_E0 - c) / d
    x0 = float(np.clip(x0, -0.999999, 0.999999))
    theta0 = np.arccos(x0)

    alpha = np.cos(np.arange(m+1) * theta0)
    if use_jackson:
        g = jackson_weights(m)
        alpha = alpha * g

    # Helper: matvec with Htilde = (H - c I)/d
    def Htilde_dot(v):
        Hv = H @ v   # works for dense or sparse
        return (Hv - c * v) / d

    # 3) Normalize random start vector if not already normalized
    v0 /= norm(v0)

    # 4) Chebyshev recursion
    t0 = v0
    t1 = Htilde_dot(v0)

    filt = alpha[0] * t0 + alpha[1] * t1

    tkm1 = t0
    tk   = t1

    for k in range(2, m+1):
        tkp1 = 2.0 * Htilde_dot(tk) - tkm1
        filt = filt + alpha[k] * tkp1
        tkm1, tk = tk, tkp1

    # 5) Normalize and Rayleigh quotient
    filt_norm = norm(filt)
    if filt_norm == 0:
        raise RuntimeError("Filtered vector became zero; try different parameters.")
    filt /= filt_norm

    Hv = H @ filt
    approx_E = np.vdot(filt, Hv).real / np.vdot(filt, filt).real

    return filt, approx_E 

In [50]:
N = 12  # Number of spins
J = 1.0  # Interaction strength
h = 3.0  # Transverse field strength # this is the value in the paper. maybe try  other values too, including the critical value one (h=J=1)

#keep_qubits = [0, 4, 5]
#keep_qubits = [0, 4, 5, 11] # 4 spins rdm full rank
#keep_qubits = [0, 4, 5, 9]  # 4 spins - 2 adjacent triangular plaquettes - this is when i observe exactly 5 degenerate scars
#keep_qubits = [0, 4, 5, 9, 11]  # 5 spins - 3 adjacent triangular plaquettes
keep_qubits = [2, 4, 5, 8, 9] # 5 spins - pentagon around 0

trace_qubits = [i for i in range(N) if i not in keep_qubits]


# Assuming transverse_field_ising is defined and returns a sparse Hermitian matrix
H = transverse_field_ising_icosahedral(N, J, h)

#print(f"Hamiltonian shape: {H.shape}")
#print(f"Non-zero elements in H: {H.nnz}")

# Define the C5 permutation mapping for your case - around axis through vert 9 and 10
perm = {
    0: 2,
    2: 7,
    7: 11,
    11: 5,
    5: 0,
    1: 3,
    3: 4,
    4: 8,
    8: 6,
    6: 1,
    9: 9,   # Fixed
    10: 10  # Fixed
}

In [51]:
eigenvalues, eigenvectors = eigh(H.toarray())

In [122]:
# build a single vector in irrep r (project a single computational basis state)
def make_irrep_from_basis(basis_idx, r, perm_arr, N):
    n = 2**N
    e = np.zeros(n, dtype=np.complex128)
    e[basis_idx] = 1.0
    omega = np.exp(2j * np.pi / 5)
    v = np.zeros_like(e)
    for t in range(5):
        v += (omega**(-r * t)) * apply_perm_power(e, perm_arr, t)
    v /= np.linalg.norm(v)
    return v

# choose irrep label r in {0,1,2,3,4} and a single computational basis index
r = 4        # pick the irrep you want
basis_idx = 0  # pick one basis state (change if you prefer another starting config)

v0 = make_irrep_from_basis(basis_idx, r, perm_arr, N)

# quick check: applying permutation should multiply by omega**r (within numerical tol)
omega = np.exp(2j * np.pi / 5)
uv = apply_perm_once(v0, perm_arr)
print("max err of U v0 - omega^r v0:", np.max(np.abs(uv - (omega**r) * v0)))

max err of U v0 - omega^r v0: 1.175570504584946


In [123]:
#v0 = np.random.randn(H.shape[0])
target_E0 = -6.0
vc, Ec = chebyshev_filter_numpy_v0(H, v0, Emin=-37.9456425, Emax=41.28675302, target_E0=target_E0, m=10000, pad=0.05, use_jackson=True)
print(Ec, np.count_nonzero(np.abs(vc) > 1e-10))
vc1, Ec1 = chebyshev_filter_numpy_v0(H, vc, Emin=-37.9456425, Emax=41.28675302, target_E0=target_E0, m=10000, pad=0.05, use_jackson=True)
print(Ec1, np.count_nonzero(np.abs(vc1) > 1e-10))
vc2, Ec2 = chebyshev_filter_numpy_v0(H, vc1, Emin=-37.9456425, Emax=41.28675302, target_E0=target_E0, m=10000, pad=0.05, use_jackson=True)
print(Ec2, np.count_nonzero(np.abs(vc2) > 1e-10))
vc3, Ec3 = chebyshev_filter_numpy_v0(H, vc2, Emin=-37.9456425, Emax=41.28675302, target_E0=target_E0, m=10000, pad=0.05, use_jackson=True)
print(Ec3, np.count_nonzero(np.abs(vc3) > 1e-10))
vc4, Ec4 = chebyshev_filter_numpy_v0(H, vc3, Emin=-37.9456425, Emax=41.28675302, target_E0=target_E0, m=10000, pad=0.05, use_jackson=True)
print(Ec4, np.count_nonzero(np.abs(vc4) > 1e-10))
vc5, Ec5 = chebyshev_filter_numpy_v0(H, vc4, Emin=-37.9456425, Emax=41.28675302, target_E0=target_E0, m=10000, pad=0.05, use_jackson=True)
print(Ec5, np.count_nonzero(np.abs(vc5) > 1e-10))

-35.99898001934579 2048
-35.99793883668962 2048


KeyboardInterrupt: 

In [ ]:
Et = eigenvalues[1527]
vt = eigenvectors[:,1527]
print(Et, np.count_nonzero(np.abs(vt) > 1e-10))

In [108]:
print((np.abs(np.dot(vt.conj(), vc4)))**2)

0.9999999999999998
